### Imports/Device/Seed/Tensorboard

In [1]:
import torch
import torch.nn as nn
import torch_directml
import torchmetrics
import torch.optim as optim
import torchmetrics.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from tqdm import trange, tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')
#AMD GPU Acceleration
device = torch_directml.device(torch_directml.default_device())

C:\Users\cafes\Envs\torch-dml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
set_seed()

In [4]:
train_writer = SummaryWriter('.\\runs\\train\\ann\\')
val_writer = SummaryWriter('.\\runs\\validate\\ann\\')

### Choose One

#### Read in train/test

In [10]:
train_df = pd.read_csv('train.csv', index_col='Time')
test_df = pd.read_csv('test.csv', index_col='Time')
train_df

,AccV,AccML,AccAP,Medication,Age,Sex,YearsSinceDx,Test_Visit,Event
Time,,,,,,,,,
0,-0.416921,0.680190,-0.585780,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
1,-0.420816,0.678475,-0.578919,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
2,-0.419886,0.676579,-0.580286,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
3,-0.413896,0.674399,-0.594393,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
4,-0.403594,0.674941,-0.607099,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
...,...,...,...,...,...,...,...,...,...
136,-0.230050,0.202884,-1.195147,-1.154029,-0.487734,0.581678,1.891825,-0.684845,0
137,-0.219754,0.198519,-1.206719,-1.154029,-0.487734,0.581678,1.891825,-0.684845,0
138,-0.210724,0.204065,-1.211322,-1.154029,-0.487734,0.581678,1.891825,-0.684845,0


In [11]:
test_df

,AccV,AccML,AccAP,Medication,Age,Sex,YearsSinceDx,Test_Visit
Time,,,,,,,,
0,-9.541841,0.572790,-1.409475,0.86653,-0.110647,-1.719166,-0.182777,0.648921
1,-9.544045,0.570599,-1.436634,0.86653,-0.110647,-1.719166,-0.182777,0.648921
2,-9.537242,0.568221,-1.425318,0.86653,-0.110647,-1.719166,-0.182777,0.648921
3,-9.539138,0.570690,-1.411444,0.86653,-0.110647,-1.719166,-0.182777,0.648921
4,-9.548736,0.568310,-1.425458,0.86653,-0.110647,-1.719166,-0.182777,0.648921
...,...,...,...,...,...,...,...,...
281683,-0.896122,-0.228349,0.397413,0.86653,1.272005,-1.719166,-0.342362,-0.792988
281684,-0.898800,-0.228733,0.398382,0.86653,1.272005,-1.719166,-0.342362,-0.792988
281685,-0.898516,-0.227567,0.400168,0.86653,1.272005,-1.719166,-0.342362,-0.792988


In [5]:
df = pd.read_csv('downsample30.csv')
test_df = pd.read_csv('test_downsample.csv')

In [6]:
df

,Id,Time,AccV,AccML,AccAP,Subject,Visit,Test,Medication,Age,Sex,YearsSinceDx,Event
0,003f117e14,0,-0.420712,0.673434,-0.591752,4dc2f8,3,2.0,1,68,0,9.0,0
1,003f117e14,1,-0.424601,0.671724,-0.584907,4dc2f8,3,2.0,1,68,0,9.0,0
2,003f117e14,2,-0.423672,0.669832,-0.586271,4dc2f8,3,2.0,1,68,0,9.0,0
3,003f117e14,3,-0.417690,0.667659,-0.600345,4dc2f8,3,2.0,1,68,0,9.0,0
4,003f117e14,4,-0.407401,0.668199,-0.613021,4dc2f8,3,2.0,1,68,0,9.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
376340,ffda8fadfd,136,-0.234080,0.197408,-1.199697,7fcee9,20,1.0,0,65,1,22.0,0
376341,ffda8fadfd,137,-0.223797,0.193055,-1.211242,7fcee9,20,1.0,0,65,1,22.0,0
376342,ffda8fadfd,138,-0.214779,0.198586,-1.215835,7fcee9,20,1.0,0,65,1,22.0,0
376343,ffda8fadfd,139,-0.213866,0.198736,-1.214870,7fcee9,20,1.0,0,65,1,22.0,0


In [7]:
train_df = df.drop(columns=['Id', 'Subject']).set_index('Time')
train_df

,AccV,AccML,AccAP,Visit,Test,Medication,Age,Sex,YearsSinceDx,Event
Time,,,,,,,,,,
0,-0.420712,0.673434,-0.591752,3,2.0,1,68,0,9.0,0
1,-0.424601,0.671724,-0.584907,3,2.0,1,68,0,9.0,0
2,-0.423672,0.669832,-0.586271,3,2.0,1,68,0,9.0,0
3,-0.417690,0.667659,-0.600345,3,2.0,1,68,0,9.0,0
4,-0.407401,0.668199,-0.613021,3,2.0,1,68,0,9.0,0
...,...,...,...,...,...,...,...,...,...,...
136,-0.234080,0.197408,-1.199697,20,1.0,0,65,1,22.0,0
137,-0.223797,0.193055,-1.211242,20,1.0,0,65,1,22.0,0
138,-0.214779,0.198586,-1.215835,20,1.0,0,65,1,22.0,0


In [8]:
test_df

,Time,AccV,AccML,AccAP,Id,Subject,Visit,Test,Medication,Age,Sex,YearsSinceDx
0,0,-9.533939,0.566322,-1.413525,003f117e14,4dc2f8,3,2.0,1,68,0,9.0
1,1,-9.536140,0.564137,-1.440621,003f117e14,4dc2f8,3,2.0,1,68,0,9.0
2,2,-9.529345,0.561765,-1.429331,003f117e14,4dc2f8,3,2.0,1,68,0,9.0
3,3,-9.531239,0.564227,-1.415490,003f117e14,4dc2f8,3,2.0,1,68,0,9.0
4,4,-9.540825,0.561854,-1.429471,003f117e14,4dc2f8,3,2.0,1,68,0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...
286365,281683,-0.899299,-0.232668,0.389147,02ab235146,e1f62e,2,0.0,1,79,0,8.0
286366,281684,-0.901973,-0.233051,0.390114,02ab235146,e1f62e,2,0.0,1,79,0,8.0
286367,281685,-0.901690,-0.231888,0.391896,02ab235146,e1f62e,2,0.0,1,79,0,8.0
286368,281686,-0.899576,-0.232430,0.391106,02ab235146,e1f62e,2,0.0,1,79,0,8.0


In [9]:
test = test_df.drop(columns=['Id', 'Subject']).set_index('Time')
test

,AccV,AccML,AccAP,Visit,Test,Medication,Age,Sex,YearsSinceDx
Time,,,,,,,,,
0,-9.533939,0.566322,-1.413525,3,2.0,1,68,0,9.0
1,-9.536140,0.564137,-1.440621,3,2.0,1,68,0,9.0
2,-9.529345,0.561765,-1.429331,3,2.0,1,68,0,9.0
3,-9.531239,0.564227,-1.415490,3,2.0,1,68,0,9.0
4,-9.540825,0.561854,-1.429471,3,2.0,1,68,0,9.0
...,...,...,...,...,...,...,...,...,...
281683,-0.899299,-0.232668,0.389147,2,0.0,1,79,0,8.0
281684,-0.901973,-0.233051,0.390114,2,0.0,1,79,0,8.0
281685,-0.901690,-0.231888,0.391896,2,0.0,1,79,0,8.0


### Dataset/DataLoader

In [12]:
#Pytorch Dataset
class ParkinsonDataset(Dataset):
    def __init__(self, df, goal):
        super().__init__()
        self.data = df.to_numpy()
        self.goal = goal

    # Return length of sample
    def __len__(self):
        return self.data.shape[0]
        
    # Get data at idx
    def __getitem__(self, idx):
        if self.goal == 'train':
            features = torch.tensor(self.data[idx, :-1], dtype=torch.float32)
            label = torch.tensor(self.data[idx, -1], dtype=torch.float32)
            features = features.to(device)
            label = label.to(device)
        else:
            features = torch.tensor(self.data[idx], dtype=torch.float32)
            features = features.to(device)
            return features

        return features, label

In [13]:
def train_val_split(df):
    #Split the training dataset into training/validation sets
    #80% Training - 20% Validation
    train_size = int(.8 * len(df))
    val_size = len(df)-train_size
    train_data, val_data = torch.utils.data.random_split(df, [train_size, val_size])

    #Instantiate DataLoaders
    train_loader = DataLoader(train_data,
                              batch_size=1024,
                              shuffle=False)

    val_loader = DataLoader(val_data,
                            batch_size=1024,
                            shuffle=False)
    
    return train_loader, val_loader

In [14]:
train = ParkinsonDataset(train_df, goal='train')
train_loader, val_loader = train_val_split(train)

In [ ]:
test = test_df.drop(columns='Id')
test = ParkinsonDataset(test, goal='test')
test_loader = DataLoader(test, 
                         batch_size=1024, 
                         shuffle=False)

In [15]:
metric = torchmetrics.AveragePrecision(task='multiclass', num_classes=4, average='weighted', thresholds=4).to(device)

### Overfit Function

In [16]:
#Function to determine whether the model is training appropriately
def overfit_batch(model, criterion, optimizer, batch, epochs):
    model = model.to(device)
    
    for epoch in range(epochs):
        metric.reset()
        loss_total = 0.0
        precision_total = 0.0
        for _ in range(len(batch)):
            features, label = batch
            features = features.to(device)
            label = label.to(device)
            
            optimizer.zero_grad()
            logits = model(features)
            loss = criterion(logits, label.long())
            ypred = torch.softmax(logits, dim=1)
            metric(ypred, label.long())
            precision=metric.compute()
            
            loss.backward()
            optimizer.step()

            loss_total += loss.item()
            precision_total += precision.item()

        avg_loss = loss_total / len(batch)
        avg_precision = precision_total / len(batch)
        if epoch %10 == 9:
            print(f'Epoch [{epoch + 1}/{epochs}] | Avg Loss: {avg_loss:.4f} | Avg Precision: {(avg_precision * 100.0):.2f}')

### Train | Validate | Predict

In [17]:
def predict(model, dataloader):
    predictions = []
    with tqdm(dataloader, colour='yellow') as pbar:
        for features in pbar:
            features = features.to(device)
            preds = model(features)
            preds = torch.argmax(preds, dim=1)
            predictions.extend(preds.cpu().numpy())
    
    return predictions

In [18]:
#Validate one epoch
def validate(model, criterion, dataloader, epoch):
    model.eval()
    metric.reset()
    
    loss_total = 0.0
    precision_total = 0.0
    
    with tqdm(dataloader, desc='Validate', total=len(dataloader), unit='batch', colour='blue') as pbar:
        with torch.no_grad():
            for batch in pbar:
                features, label = batch
                features = features.to(device)
                label = label.to(device)

                logits = model(features)               
                #Update running loss and precision
                loss = criterion(logits, label.long())
                ypred = torch.softmax(logits, dim=1)
                metric(ypred, label.long())
                precision=metric.compute()
                
                precision_total += precision.item()
                loss_total += loss.item()

                #Update the tqdm progress bar description with loss and precision
                pbar.set_postfix_str(s=f'Loss: {loss:.4f} | Precision: {precision*100:.2f}%')
                
        avg_loss = loss_total / len(dataloader)
        avg_precision = precision_total / len(dataloader)
        
    model.train()    
    return avg_loss, avg_precision

In [19]:
def train_validate(model, criterion, optimizer, scheduler, epochs=100, train_loader=train_loader, val_loader=val_loader, patience=5):
    model = model.train()
    best_precision = None
    min_delta = 0
    counter = 0
    
    for epoch in range(epochs):
        metric.reset()
        loss_total = 0.0
        precision_total = 0.0
        with tqdm(train_loader, desc='Train', total=len(train_loader), unit='batch', colour='green') as pbar:
            for batch in pbar:
                #Set features/targets to device
                features, label = batch
                features = features.to(device)
                label = label.to(device)

                #Predict labels
                optimizer.zero_grad()                
                logits = model(features)
                loss = criterion(logits, label.long())
                ypred = torch.softmax(logits, dim=1)
                metric(ypred, label.long())
                precision=metric.compute()
                
                #Tensorboard
                train_writer.add_scalar('Train-Loss', loss, epoch)
                train_writer.add_scalar('Train-Precision', precision, epoch)
                
                pbar.set_postfix_str(s=f'Loss: {loss:.4f} | Precision: {precision*100:.2f}%')

                #Adjust gradient
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                #Accumulate total loss and precision
                loss_total += loss.item()
                precision_total += precision.item()

            avg_loss = loss_total / len(train_loader)
            avg_precision = precision_total / len(train_loader)
         
        #Report training loss/precision
        print(f'Epoch {epoch + 1} | Avg Loss: {avg_loss:.4f} | Avg Precision: {(avg_precision * 100.0):.2f}%')

        if epoch%3==2:
            #Validate one epoch and update lr scheduler
            avg_val_loss, avg_val_precision = validate(model, criterion, val_loader, epoch)
            scheduler.step(avg_val_precision)
        
            #Early Stopping
            if best_precision == None:
                best_precision = avg_val_precision
            elif(best_precision - avg_val_precision < min_delta):
                best_precision = avg_val_precision
                counter = 0
            elif(best_precision - avg_val_precision > min_delta):
                counter += 1
                if(counter >= patience):
                    print(f'Early Stopping: Precision stagnated or decreased too many times. Best Validation: {(best_precision*100.0):.2f}%')
                    break

            print(f'Epoch {epoch + 1} | Avg Loss: {avg_val_loss:.4f} | Avg Precision: {(avg_val_precision * 100.0):.2f}% | Patience: {counter}')

    train_writer.flush()
    train_writer.close()
    val_writer.flush()
    val_writer.close()
    print(f'Training Complete - Best Validate Precision: {(best_precision*100.0):.2f}%')
    
    return model

### Model Architecture

In [22]:
#Neural Network model architecture
class Ann(nn.Module):
    def __init__(self):
        super(Ann, self).__init__()
        self.fc1 = nn.Linear(8, 300)
        self.fc2 = nn.Linear(300, 150)
        self.fc3 = nn.Linear(150, 75)
        self.fc4 = nn.Linear(75, 4)

        #Batch Normalization Layers
        self.bn1 = nn.BatchNorm1d(300)
        self.bn2 = nn.BatchNorm1d(150)
        self.bn3 = nn.BatchNorm1d(75)

        #Add Dropout Layers
        self.dropout = nn.Dropout(p=0.2)
        
        #Initialize He Kaiming weights
        nn.init.kaiming_uniform_(self.fc1.weight)
        nn.init.kaiming_uniform_(self.fc2.weight)
        nn.init.kaiming_uniform_(self.fc3.weight)
        nn.init.kaiming_uniform_(self.fc4.weight)
        
    #Forward Propogation
    def forward(self, x):
        #Pass through the first layer
        x = self.fc1(x)
        x = self.bn1(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)

        #Second layer pass
        x = self.fc2(x)
        x = self.bn2(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        
        #Third layer pass
        x = self.fc3(x)
        x = self.bn3(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        
        #output
        x = self.fc4(x)
        return x

### Overfitting

In [54]:
#overfit batch
batch = next(iter(train_loader))
model = Ann()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr = .09)
epochs=100
overfit_batch(model, criterion, optimizer, batch, epochs)

Epoch [10/100] | Avg Loss: 0.6763 | Avg Precision: 65.29
Epoch [20/100] | Avg Loss: 0.6033 | Avg Precision: 67.34
Epoch [30/100] | Avg Loss: 0.5772 | Avg Precision: 71.14
Epoch [40/100] | Avg Loss: 0.5527 | Avg Precision: 70.02
Epoch [50/100] | Avg Loss: 0.5286 | Avg Precision: 73.23
Epoch [60/100] | Avg Loss: 0.5257 | Avg Precision: 73.39
Epoch [70/100] | Avg Loss: 0.5000 | Avg Precision: 73.44
Epoch [80/100] | Avg Loss: 0.4869 | Avg Precision: 73.61
Epoch [90/100] | Avg Loss: 0.4944 | Avg Precision: 73.97
Epoch [100/100] | Avg Loss: 0.4830 | Avg Precision: 75.86


# Training

## Subset

### Batch 1024 

## Downsample

### 10-30-23

#### Model 2 75-35 Nodes

In [20]:
model = Ann()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.03)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2)
epochs = 100

model2 = train_validate(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader)
torch.save(model2.state_dict(), '.\\models\\downsample\\ann\\model2.pth')

Train: 100%|████████████████████████| 295/295 [00:37<00:00,  7.86batch/s, Loss: 0.4071 | Precision: 72.98%]


Epoch 1 | Avg Loss: 0.5206 | Avg Precision: 70.61%


Train: 100%|████████████████████████| 295/295 [00:37<00:00,  7.87batch/s, Loss: 0.3973 | Precision: 76.04%]


Epoch 2 | Avg Loss: 0.4634 | Avg Precision: 75.49%


Train: 100%|████████████████████████| 295/295 [00:38<00:00,  7.73batch/s, Loss: 0.3072 | Precision: 77.02%]


...

Epoch 71 | Avg Loss: 0.3490 | Avg Precision: 83.38%


Train: 100%|████████████████████████| 295/295 [00:33<00:00,  8.75batch/s, Loss: 0.2721 | Precision: 83.46%]


Epoch 72 | Avg Loss: 0.3489 | Avg Precision: 83.38%


Validate: 100%|███████████████████████| 74/74 [00:08<00:00,  9.09batch/s, Loss: 0.3770 | Precision: 83.49%]

Early Stopping: Precision stagnated or decreased too many times. Best Validation: 83.49%
Training Complete


### 10-31-23

#### Model 3 - Batch 512

In [23]:
model = Ann()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.03)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2)
epochs = 100

model3 = train_validate(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader)
torch.save(model3.state_dict(), '.\\models\\downsample\\ann\\model3.pth')

Train: 100%|████████████████████████| 589/589 [00:43<00:00, 13.40batch/s, Loss: 0.3871 | Precision: 73.61%]


Epoch 1 | Avg Loss: 0.5078 | Avg Precision: 71.51%


Train: 100%|████████████████████████| 589/589 [00:44<00:00, 13.16batch/s, Loss: 0.4191 | Precision: 76.23%]


Epoch 2 | Avg Loss: 0.4620 | Avg Precision: 75.77%


Train: 100%|████████████████████████| 589/589 [00:44<00:00, 13.28batch/s, Loss: 0.5046 | Precision: 76.87%]


...

Epoch 99 | Avg Loss: 0.3184 | Avg Precision: 84.71%


Validate: 100%|█████████████████████| 148/148 [00:09<00:00, 16.21batch/s, Loss: 0.1024 | Precision: 84.94%]


Epoch 99 | Avg Loss: 0.3166 | Avg Precision: 84.88% | Patience: 0


Train: 100%|████████████████████████| 589/589 [00:38<00:00, 15.16batch/s, Loss: 0.3081 | Precision: 84.77%]

Epoch 100 | Avg Loss: 0.3183 | Avg Precision: 84.71%
Training Complete


### 11-1-23

#### Model 4 - Batch 250

In [25]:
model = Ann()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.03)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2)
epochs = 100

model4 = train_validate(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader)
torch.save(model4.state_dict(), '.\\models\\downsample\\ann\\model4.pth')

Train: 100%|██████████████████████| 1205/1205 [00:53<00:00, 22.68batch/s, Loss: 0.4562 | Precision: 73.76%]


Epoch 1 | Avg Loss: 0.5056 | Avg Precision: 72.14%


Train: 100%|██████████████████████| 1205/1205 [00:53<00:00, 22.60batch/s, Loss: 0.4732 | Precision: 75.83%]


Epoch 2 | Avg Loss: 0.4667 | Avg Precision: 75.56%


Train: 100%|██████████████████████| 1205/1205 [00:53<00:00, 22.62batch/s, Loss: 0.5057 | Precision: 76.65%]


...

Epoch 35 | Avg Loss: 0.3566 | Avg Precision: 82.86%


Train: 100%|██████████████████████| 1205/1205 [00:48<00:00, 24.94batch/s, Loss: 0.3344 | Precision: 82.81%]


Epoch 36 | Avg Loss: 0.3566 | Avg Precision: 82.90%


Validate: 100%|███████████████████████| 74/74 [00:08<00:00,  9.11batch/s, Loss: 0.2981 | Precision: 82.77%]

Early Stopping: Precision stagnated or decreased too many times. Best Validation: 83.04%
Training Complete


### 11-14-23

#### Model 5

In [23]:
model = Ann()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=0, min_lr=.0001, verbose=True)

model5 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model5.state_dict(), '.\\models\\downsample\\ann\\model5.pth')

Train: 100%|████████████████████████| 295/295 [00:41<00:00,  7.08batch/s, Loss: 0.3615 | Precision: 75.65%]


Epoch 1 | Avg Loss: 0.4783 | Avg Precision: 73.27%


Train: 100%|████████████████████████| 295/295 [00:40<00:00,  7.34batch/s, Loss: 0.2156 | Precision: 78.90%]


Epoch 2 | Avg Loss: 0.4181 | Avg Precision: 78.27%


Train: 100%|████████████████████████| 295/295 [00:40<00:00,  7.36batch/s, Loss: 0.2195 | Precision: 79.86%]


...

Epoch 74 | Avg Loss: 0.2548 | Avg Precision: 87.93%


Train: 100%|████████████████████████| 295/295 [00:35<00:00,  8.24batch/s, Loss: 0.0304 | Precision: 88.09%]


Epoch 75 | Avg Loss: 0.2543 | Avg Precision: 87.93%


Validate: 100%|███████████████████████| 74/74 [00:08<00:00,  8.51batch/s, Loss: 0.3348 | Precision: 85.87%]

Epoch 00025: reducing learning rate of group 0 to 2.8243e-03.
Early Stopping: Precision stagnated or decreased too many times. Best Validation: 85.90%
Training Complete - Best Validate Precision: 85.90%


### Predict | Submit

In [ ]:
ypreds = predict(model1, test_loader)
test_df['Prediction'] = ypreds

In [ ]:
sub = pd.DataFrame()
temp = test_df.copy()
temp["Id"] = temp.apply(lambda x : str(x.Id) + "_" + str(x.Time), axis = 1)
temp['StartHesitation'] = np.where(temp['Prediction'] == 1, 1, 0)
temp['Turn'] = np.where(temp['Prediction'] == 2, 1, 0)
temp['Walking'] = np.where(temp['Prediction'] == 3, 1, 0)
temp = temp[['Id', 'StartHesitation', 'Turn', 'Walking']]
sub = pd.concat([sub, temp])

In [26]:
ypreds = predict(model1, test_loader)

100%|████████████████████████████████████████████████████████████████████| 287/287 [00:14<00:00, 19.95it/s]


In [27]:
ypreds.shape

(286370,)

In [28]:
test_df['Prediction'] = ypreds
test_df

,Time,AccV,AccML,AccAP,Id,Subject,Visit,Test,Medication,Age,Sex,YearsSinceDx,Prediction
0,0,-9.533939,0.566322,-1.413525,003f117e14,4dc2f8,3,2.0,1,68,0,9.0,0.0
1,1,-9.536140,0.564137,-1.440621,003f117e14,4dc2f8,3,2.0,1,68,0,9.0,0.0
2,2,-9.529345,0.561765,-1.429331,003f117e14,4dc2f8,3,2.0,1,68,0,9.0,0.0
3,3,-9.531239,0.564227,-1.415490,003f117e14,4dc2f8,3,2.0,1,68,0,9.0,0.0
4,4,-9.540825,0.561854,-1.429471,003f117e14,4dc2f8,3,2.0,1,68,0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
286365,281683,-0.899299,-0.232668,0.389147,02ab235146,e1f62e,2,0.0,1,79,0,8.0,0.0
286366,281684,-0.901973,-0.233051,0.390114,02ab235146,e1f62e,2,0.0,1,79,0,8.0,0.0
286367,281685,-0.901690,-0.231888,0.391896,02ab235146,e1f62e,2,0.0,1,79,0,8.0,0.0
286368,281686,-0.899576,-0.232430,0.391106,02ab235146,e1f62e,2,0.0,1,79,0,8.0,0.0


In [54]:
test_df.Prediction.value_counts()

Prediction
0.0    286370
Name: count, dtype: int64

In [38]:
sub = pd.DataFrame()
temp = test_df.copy()
temp["Id"] = temp.apply(lambda x : str(x.Id) + "_" + str(x.Time), axis = 1)
temp['StartHesitation'] = np.where(temp['Prediction'] == 1, 1, 0)
temp['Turn'] = np.where(temp['Prediction'] == 2, 1, 0)
temp['Walking'] = np.where(temp['Prediction'] == 3, 1, 0)
temp = temp[['Id', 'StartHesitation', 'Turn', 'Walking']]
sub = pd.concat([sub, temp])

In [39]:
sub.head()

,Id,StartHesitation,Turn,Walking
0,003f117e14_0,0,0,0
1,003f117e14_1,0,0,0
2,003f117e14_2,0,0,0
3,003f117e14_3,0,0,0
4,003f117e14_4,0,0,0


In [40]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286370 entries, 0 to 286369
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Id               286370 non-null  object
 1   StartHesitation  286370 non-null  int32 
 2   Turn             286370 non-null  int32 
 3   Walking          286370 non-null  int32 
dtypes: int32(3), object(1)
memory usage: 5.5+ MB
